In [1]:
cd ..

/home/andv/important/chatbot_vnuis


/home/andv/important/chatbot_vnuis/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from sentence_transformers import SentenceTransformer, util
import torch
import numpy as np
import pandas as pd
import textwrap
import time
import sys
import locale
import os
from dotenv import load_dotenv
import json

load_dotenv()
# Load models and data once
embedding_path = "Model/vnuis_embedding_bge_final"
embedding_model = SentenceTransformer(
    model_name_or_path=embedding_path,
    model_kwargs={"torch_dtype": "bfloat16"},
    device="cuda",
    trust_remote_code=True,
)
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

rerank_tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-reranker-v2-m3")
rerank_model = AutoModelForSequenceClassification.from_pretrained(
    "BAAI/bge-reranker-v2-m3", torch_dtype=torch.bfloat16, device_map="cuda:0"
)
rerank_model.eval()


# Load embeddings and data once
def load_embeddings(embeddings_path, device):
    try:
        text_chunks_and_embedding_df = pd.read_csv(embeddings_path)

        # Convert the embedding column from JSON strings to lists of floats
        text_chunks_and_embedding_df["embedding"] = text_chunks_and_embedding_df[
            "embedding"
        ].apply(json.loads)

        # Convert embeddings to PyTorch tensors
        embeddings = torch.tensor(
            np.array(text_chunks_and_embedding_df["embedding"].tolist()),
            dtype=torch.bfloat16,
        ).to(device)

        pages_and_chunks = text_chunks_and_embedding_df.to_dict(orient="records")
    except Exception as e:
        raise RuntimeError(f"Failed to load embeddings: {e}")

    return embeddings, pages_and_chunks


embeddings, pages_and_chunks = load_embeddings(
    embeddings_path="Data/Embedding.csv", device="cuda"
)


# Function to retrieve relevant resources
def retrieve_relevant_resources(
    query: str, result_embedding: int = 20, result_reranking: int = 5
):
    query_embedding = embedding_model.encode(query, convert_to_tensor=True)
    # Use cosine similarity instead of dot score
    dot_scores = util.dot_score(query_embedding, embeddings)[0]

    # Get top scores with a threshold
    scores, indices = torch.topk(input=dot_scores, k=result_embedding)

    # cosine_scores = util.pytorch_cos_sim(query_embedding, embeddings)[0]
    # scores, indices = torch.topk(input=cosine_scores, k=n_resources_to_return)
    context_items = [pages_and_chunks[i] for i in indices]

    results = [item["Relevant docs"] for item in context_items]

    # ques_results = [item["Question"] for item in context_items]
    pairs = [[query, result] for result in results]

    with torch.no_grad():
        inputs = rerank_tokenizer(
            pairs, padding=True, truncation=True, return_tensors="pt", max_length=526
        )
        inputs = {
            key: value.to("cuda") for key, value in inputs.items()
        }  # Move all inputs to the same device as the model

        # Compute scores
        rerank_scores = rerank_model(**inputs, return_dict=True).logits.view(
            -1,
        )

        top_scores, top_indices = torch.topk(rerank_scores, k=result_reranking)
        rerank_result = [results[i] for i in top_indices]
    return results, scores, top_scores, rerank_result

/home/andv/important/chatbot_vnuis/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [36]:
query = "Bạn tên là gì?"
results, scores, top_scores, rerank_result = retrieve_relevant_resources(
    query=query, result_embedding=20, result_reranking=5
)

In [37]:
results

['Chào bạn tôi là VNU-IS Chatbot, bạn cần hỗ trợ gì ạ?',
 'Tôi là VNU-IS Chatbot, hỗ trợ bạn giải đáp những thắc mắc liên quan đến Trường Quốc Tế - Đại học Quốc gia Hà Nội',
 'Nếu mỗi nốt nhạc thể hiện một khung bậc của cảm xúc thì ISHUFFLE chính là bản giao hưởng đó. Đến với ISHUFFLE bạn sẽ được trải nghiệm bản nhạc đó bằng cách chân thực nhất có thể. Ngay bây giờ, hãy xem, đến với CLB của chúng mình các bạn sẽ được trải nghiệm những gì nha\n\nChúng mình tên gì?\n\n– Tên Tiếng Việt: Câu lạc bộ Âm nhạc ISHUFFLE\n\n– Tên Tiếng Anh: ISHUFFLE Music Club\n\nLĩnh vực hoạt động là gì?\nĐúng như cái tên, Âm nhạc là lĩnh vực hoạt động chính của chúng mình. Ngoài ca hát, chúng mình còn thêm những vị trí khác như guitar, piano, drum và rap.\nVề định hướng phát triển?\nVới sự phát triển không ngừng nghỉ, ISHUFFLE luôn cố gắng tạo ra những sân chơi cho những bạn trẻ đam mê và yêu thích âm nhạc. Là nơi cầu nối gắn kết các mảnh ghép rời rạc thành một bức tranh đa sắc màu, thành một “gia đình” đoàn k

In [38]:
scores

tensor([0.3867, 0.3691, 0.2949, 0.2871, 0.2676, 0.2656, 0.2578, 0.2539, 0.2539,
        0.2500, 0.2500, 0.2441, 0.2422, 0.2412, 0.2373, 0.2373, 0.2354, 0.2334,
        0.2305, 0.2295], device='cuda:0', dtype=torch.bfloat16)

In [39]:
top_scores

tensor([-2.9688, -5.2188, -5.6562, -5.7500, -6.1875], device='cuda:0',
       dtype=torch.bfloat16)

In [40]:
rerank_result

['Nếu mỗi nốt nhạc thể hiện một khung bậc của cảm xúc thì ISHUFFLE chính là bản giao hưởng đó. Đến với ISHUFFLE bạn sẽ được trải nghiệm bản nhạc đó bằng cách chân thực nhất có thể. Ngay bây giờ, hãy xem, đến với CLB của chúng mình các bạn sẽ được trải nghiệm những gì nha\n\nChúng mình tên gì?\n\n– Tên Tiếng Việt: Câu lạc bộ Âm nhạc ISHUFFLE\n\n– Tên Tiếng Anh: ISHUFFLE Music Club\n\nLĩnh vực hoạt động là gì?\nĐúng như cái tên, Âm nhạc là lĩnh vực hoạt động chính của chúng mình. Ngoài ca hát, chúng mình còn thêm những vị trí khác như guitar, piano, drum và rap.\nVề định hướng phát triển?\nVới sự phát triển không ngừng nghỉ, ISHUFFLE luôn cố gắng tạo ra những sân chơi cho những bạn trẻ đam mê và yêu thích âm nhạc. Là nơi cầu nối gắn kết các mảnh ghép rời rạc thành một bức tranh đa sắc màu, thành một “gia đình” đoàn kết. Không có điểm dừng hay khái niệm cụ thể nào cho âm nhạc. Chúng mình luôn trân trọng những tài năng âm nhạc trẻ và sẵn sàng đồng hành cùng các bạn để phát triển niềm đam m